##### Copyright 2018 The TensorFlow Authors.

Jun:
This notebook is based on the tensorflow tutorial "Text Classification with preprocessing". I did not change the network architecture. Basically, I trained the network on IMDb dataset and tested it on the Tieto dataset.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Text classification with preprocessed text: Movie reviews

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


This notebook classifies movie reviews as *positive* or *negative* using the text of the review. This is an example of *binary*—or two-class—classification, an important and widely applicable kind of machine learning problem.

We'll use the [IMDB dataset](https://www.tensorflow.org/datasets/catalog/imdb_reviews) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are *balanced*, meaning they contain an equal number of positive and negative reviews.

This notebook uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For a more advanced text classification tutorial using `tf.keras`, see the [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/).

## Setup

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [5]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [75]:
from tensorflow import keras
# import tfds-nightly
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
print(tf.__version__)

2.0.0-dev20191002


<a id="download"></a>

## Download the IMDB dataset

The IMDB movie reviews dataset comes packaged in `tfds`. It has already been preprocessed so that the reviews (sequences of words) have been converted to sequences of integers, where each integer represents a specific word in a dictionary.

The following code downloads the IMDB dataset to your machine (or uses a cached copy if you've already downloaded it):

To encode your own text see the [Loading text tutorial](../load_data/text.ipynb)

In [7]:
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)

<a id="encoder"></a>

## Try the encoder

 The dataset `info` includes the text encoder (a `tfds.features.text.SubwordTextEncoder`).

In [8]:
encoder = info.features['text'].encoder

In [9]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


This text encoder will reversibly encode any string:

In [10]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('The original string: "{}"'.format(original_string))

assert original_string == sample_string

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


The encoder encodes the string by breaking it into subwords or characters if the word is not in its dictionary. So the more a string resembles the dataset, the shorter the encoded representation will be.

In [11]:
for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder.decode([ts])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


## Explore the data

Let's take a moment to understand the format of the data. The dataset comes preprocessed: each example is an array of integers representing the words of the movie review. 

The text of reviews have been converted to integers, where each integer represents a specific word-piece in the dictionary. 

Each label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

Here's what the first review looks like:

In [12]:
for train_example, train_label in train_data.take(2):
  print('Encoded text:', train_example.numpy())
  print('Label:', train_label.numpy())

Encoded text: [ 249    4  277  309  560    6 6639 4574    2   12   31 7759 3525 2128
   93 2306   43 2312 2527    6   30 1334 8044   24   10   16   10   17
  977   30  815 3339   41  841 7911  376 7974 1923    6  607  219   44
  242 1603   11 4329  102 2821 1139    2  969  161   51   18    4 5844
 2820  123 4493   40    6 4571   13  117   35  289  850  455   50  460
 6359 1069  343   20    1 3733 3511 7670    3  147    4  336    2   42
   18    4 3422  409 3533  871 2836  311    5 5080 1209    3  183  117
   35 1187    5 1955   11    1  226 7745    3  183 1466 7359 7961 1466
  665    2 6854 3178 1377 6266 1447  297    2 5797   36 4740  847 8050
    2    5 1929 1631 5986   22 5541 5688    5 1036 3746 8050    3   69
  264   35    4 4224    2   26   42   18    4  474 7968    8 1626   24
   10   16   10   17  134   15    9    1  435   13    9   55  598 2357
   48   30 6611 8044    3  500    1  101    6 2793    2 6548 7961 5861
  311    9  139 5398 2270   11 3674 1642   25 7796 7961 5012   

The `info` structure contains the encoder/decoder. The encoder can be used to recover the original text:

In [13]:
encoder.decode(train_example)

"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the storyline is awesome is very unique and i'm sure u are going to like it. Jenna amazed us once more and no wonder the movie won so many awards. Her make-up and wardrobe is very very sexy and the girls on girls scene is amazing. specially the one where she looks like an angel. It's a must see and i hope u share my interests"

## Prepare the data for training

You will want to create batches of training data for your model. The reviews are all different lengths, so use `padded_batch` to zero pad the sequences while batching:

In [14]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32, tf.compat.v1.data.get_output_shapes(train_data)))
#     .padded_batch(32, train_data.output_shapes))

test_batches = (
    test_data
    .padded_batch(32, tf.compat.v1.data.get_output_shapes(train_data)))
#     .padded_batch(32, train_data.output_shapes))

Each batch will have a shape of `(batch_size, sequence_length)` because the padding is dynamic each batch will have a different length:

In [15]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch)
  print("label shape:", label_batch)
  

Batch shape: tf.Tensor(
[[ 492   72   12 ...    0    0    0]
 [  12  574    7 ...    0    0    0]
 [  12   31    7 ...    0    0    0]
 ...
 [2016  131  110 ...    0    0    0]
 [ 274   41 7066 ...    0    0    0]
 [  12  140  284 ...    0    0    0]], shape=(32, 886), dtype=int64)
label shape: tf.Tensor([0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 0 0 0 1 0 1], shape=(32,), dtype=int64)
Batch shape: tf.Tensor(
[[ 387  118 3437 ...    0    0    0]
 [3502  112   48 ...    0    0    0]
 [  12  284   14 ...    0    0    0]
 ...
 [2202  103   81 ...    0    0    0]
 [ 597  218   14 ... 3641 7975 2449]
 [  19 3920 1931 ...    0    0    0]], shape=(32, 938), dtype=int64)
label shape: tf.Tensor([1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0], shape=(32,), dtype=int64)


## Build the model

The neural network is created by stacking layers—this requires two main architectural decisions:

* How many layers to use in the model?
* How many *hidden units* to use for each layer?

In this example, the input data consists of an array of word-indices. The labels to predict are either 0 or 1. Let's build a "Continuous bag of words" style model for this problem:

Caution: This model doesn't use masking, so the zero-padding is used as part of the input, so the padding length may affect the output.  To fix this, see the [masking and padding guide](../../guide/keras/masking_and_padding).

In [16]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:

1. The first layer is an `Embedding` layer. This layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch, sequence, embedding)`.
2. Next, a `GlobalAveragePooling1D` layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.
3. This fixed-length output vector is piped through a fully-connected (`Dense`) layer with 16 hidden units.
4. The last layer is densely connected with a single output node. Using the `sigmoid` activation function, this value is a float between 0 and 1, representing a probability, or confidence level.

### Hidden units

The above model has two intermediate or "hidden" layers, between the input and output. The number of outputs (units, nodes, or neurons) is the dimension of the representational space for the layer. In other words, the amount of freedom the network is allowed when learning an internal representation.

If a model has more hidden units (a higher-dimensional representation space), and/or more layers, then the network can learn more complex representations. However, it makes the network more computationally expensive and may lead to learning unwanted patterns—patterns that improve performance on training data but not on the test data. This is called *overfitting*, and we'll explore it later.

### Loss function and optimizer

A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the `binary_crossentropy` loss function.

This isn't the only choice for a loss function, you could, for instance, choose `mean_squared_error`. But, generally, `binary_crossentropy` is better for dealing with probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.

Later, when we are exploring regression problems (say, to predict the price of a house), we will see how to use another loss function called mean squared error.

Now, configure the model to use an optimizer and a loss function:

In [17]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train the model

Train the model by passing the `Dataset` object to the model's fit function. Set the number of epochs.

In [18]:
history = model.fit(train_batches,
                    epochs=15,
                    validation_data=test_batches,
#                     steps_per_epoch = 782,
                    validation_steps=30)

Epoch 1/15
782/782 [==============================] - 4s 5ms/step - loss: 0.6827 - accuracy: 0.6194 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/15
782/782 [==============================] - 2s 3ms/step - loss: 0.6242 - accuracy: 0.7446 - val_loss: 0.5979 - val_accuracy: 0.7781
Epoch 3/15
782/782 [==============================] - 3s 3ms/step - loss: 0.5463 - accuracy: 0.8054 - val_loss: 0.5318 - val_accuracy: 0.8146
Epoch 4/15
782/782 [==============================] - 2s 3ms/step - loss: 0.4797 - accuracy: 0.8380 - val_loss: 0.4810 - val_accuracy: 0.8365
Epoch 5/15
782/782 [==============================] - 2s 3ms/step - loss: 0.4258 - accuracy: 0.8605 - val_loss: 0.4446 - val_accuracy: 0.8375
Epoch 6/15
782/782 [==============================] - 3s 3ms/step - loss: 0.3849 - accuracy: 0.8741 - val_loss: 0.4099 - val_accuracy: 0.8552
Epoch 7/15
782/782 [==============================] - 3s 3ms/step - loss: 0.3524 - accuracy: 0.8857 - val_loss: 0.3790 - val_accuracy: 0.867

## Evaluate the model

And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, lower values are better), and accuracy.

In [19]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

    782/Unknown - 2s 3ms/step - loss: 0.3098 - accuracy: 0.8811Loss:  0.30982110934222445
Accuracy:  0.88112


This fairly naive approach achieves an accuracy of about 87%. With more advanced approaches, the model should get closer to 95%.

# Jun: Test on the Tieto Dataset


In [65]:
# Read test data from Tieto
import os
length = 0
path = 'movie_review_data'
classes = ['neg', 'pos']
labels    = []
test_data_mine = []
space = ' '
for j in range(len(classes)):
  file_list = os.listdir(path+'/'+classes[j])
  for i in file_list:
    labels.append(j)
    comment = open(path+'/'+classes[j]+'/'+i).read()
    comment = comment.replace('\n',' ')
    len_temp = len(encoder.encode(comment))
    if len_temp > length:
        length = len_temp    
    test_data_mine.append(comment)
length

3983

In [66]:
# Save the tieto dataset to a 2d array. Pad each sample with 0 to the max length
reviews_mine = np.zeros((len(test_data_mine),length))
for i in range(len(test_data_mine)):
    list_tmp = encoder.encode(test_data_mine[i].strip('\n').strip())
    for j in range(len(list_tmp)):
        reviews_mine[i][j] = list_tmp[j]
labels = np.array([0 if i < 999 else 1 for i in range(1999)])


In [68]:
# Transform the tieto dataset to tf dataset object
dataset = tf.data.Dataset.from_tensor_slices((reviews_mine, labels))
for line in dataset.take(5):
  print(line)

(<tf.Tensor: shape=(3983,), dtype=float64, numpy=array([ 300., 1228.,  300., ...,    0.,    0.,    0.])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(3983,), dtype=float64, numpy=array([ 260., 7968.,   21., ...,    0.,    0.,    0.])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(3983,), dtype=float64, numpy=array([  54., 2766., 1170., ...,    0.,    0.,    0.])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(3983,), dtype=float64, numpy=array([5609., 2329., 2098., ...,    0.,    0.,    0.])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(3983,), dtype=float64, numpy=array([  72., 4908., 2041., ...,    0.,    0.,    0.])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [89]:
# Test On Tieto Dataset
mine_test_batches = (
    dataset
    .padded_batch(32, tf.compat.v1.data.get_output_shapes(train_data)))
print( model.evaluate(mine_test_batches))
predictions = model.predict(mine_test_batches)
predictions = np.array([0 if i[0] < 0.5 else 1 for i in predictions])

print ('F1 Score: {}'.format(f1_score(labels, predictions)))
    
con_matrix = confusion_matrix(labels, predictions, normalize = True)
con_matrix = con_matrix/np.sum(con_matrix,axis=1)
con_matrix


63/63 [==============================] - 0s 4ms/step - loss: 0.3954 - accuracy: 0.8429
[0.3954093569800967, 0.84292144]
F1 Score: 0.8276619099890231


array([[0.93193193, 0.068     ],
       [0.24624625, 0.754     ]])

## Create a graph of accuracy and loss over time

`model.fit()` returns a `History` object that contains a dictionary with everything that happened during training:

In [ ]:
history_dict = history.history
history_dict.keys()

There are four entries: one for each monitored metric during training and validation. We can use these to plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()


In this plot, the dots represent the training loss and accuracy, and the solid lines are the validation loss and accuracy.

Notice the training loss *decreases* with each epoch and the training accuracy *increases* with each epoch. This is expected when using a gradient descent optimization—it should minimize the desired quantity on every iteration.

This isn't the case for the validation loss and accuracy—they seem to peak after about twenty epochs. This is an example of overfitting: the model performs better on the training data than it does on data it has never seen before. After this point, the model over-optimizes and learns representations *specific* to the training data that do not *generalize* to test data.

For this particular case, we could prevent overfitting by simply stopping the training after twenty or so epochs. Later, you'll see how to do this automatically with a callback.